In [1]:
# !pip install -qU llama-index llama-parse python-dotenv

In [2]:
# !pip install -qU llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

In [1]:
# !pip show torch
# Name: llama-index
# Version: 0.10.35

# Name: llama-index-postprocessor-flag-embedding-reranker
# Version: 0.1.2

Reference notebook: [LlamaParse - A Tool for Parsing Complex Documents.ipynb](https://colab.research.google.com/drive/1IVQkSGwS5kdTiKBwz85PO6vg_WaNx15c?usp=sharing#scrollTo=pLlEWwpFqFrp)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# wd_path = "/content/drive/MyDrive/Colab Notebooks/rag-notebooks"
wd_path = ".."

In [5]:
import sys
import openai
import os
import getpass
import logging

from dotenv import load_dotenv, find_dotenv

In [6]:
def navigate_up(path, levels):
    """Navigate up `levels` directories from the given path."""
    for _ in range(levels):
        path = os.path.dirname(path)
    return path

# Get the current working directory
llamaindex_dir = os.getcwd()
# Get the parent directory
llamaindex_dir = os.path.dirname(llamaindex_dir)

sys.path.append(llamaindex_dir + "/utils")
sys.path.append(navigate_up(llamaindex_dir, 2) + "/law-sec-insights/backend")
# sys.path

In [7]:
# os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

In [8]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [9]:
# os.environ["HF_TOKEN"] = getpass.getpass("Hugging Face API Key:")

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [12]:
from llama_index.core.callbacks.global_handlers import set_global_handler

set_global_handler("simple")
# set_global_handler("langfuse")

In [13]:
# import llama_parse
# import llama_index
# import os

# llama_parse_path = os.path.dirname(llama_parse.__file__)
# llama_index_path = os.path.dirname(llama_index.__file__)

# print(llama_parse_path)
# print(llama_index_path)

# Helpers

In [14]:
from llamaindex_utils import *
from duvidas import *

# Law questions

In [15]:
current_type = "Cédula de Crédito Bancário"
for col_name, duvida_map in duvidas.items():
    print(duvida_map["Cédula de Crédito Bancário"])
    print()

Quem é o  Devedor? (Emitente)
Depois de responder, identifica a cláusula desta seção entre as cláusulas abaixo:
['EMITENTE', 'DADOS DA OPERAÇÃO DE CRÉDITO', 'AVALISTA(S)', 'DESTINAÇÃO DO CRÉDITO', 'ENCARGOS FINANCEIROS', 'FORMA DE PAGAMENTO', 'COMISSÃO FLAT', 'TARIFAS', 'IOF', 'INADIMPLEMENTO', 'HONORÁRIOS ADVOCATÍCIOS', 'PLANILHA DE CÁLCULO PARA COBRANÇA', 'AUTORIZAÇÃO PARA DÉBITO EM CONTA', 'DESPESAS CARTORÁRIAS', 'LIQUIDAÇÃO/AMORTIZAÇÃO ANTECIPADA', 'VENCIMENTO EXTRAORDINÁRIO/ANTECIPADO', 'CLAUSULA ESPECIAL DE VENCIMENTO ANTECIPADO', 'COMUNICAÇÃO DENALTERAÇÃO ESTATUTÁRIA E DE DIRIGENTES', 'SUSPENSÃO DE LIBERAÇÃO DE CREDITO', 'PRESERVAÇÃO DE DIREITOS', 'COMPENSAÇÃO DE CRÉDITOS', 'CESSÃO', 'IMPUTAÇÃO AO PAGAMENTO', 'PREVENÇÃO E COMBATE À CORRUPÇÃO', 'RESPONSABILIDADE SOCIOAMBIENTAL', 'DECLARAÇÃO ESPECIAL', 'SISTEMA DE INFORMAÇÕES DE CRÉDITO DO BANCO CENTRAL - SCR', 'CENTRAIS DE ATENDIMENTO TELEFÔNICO', 'AVALISTAS', 'FORMALIZAÇÃO', 'LOCAL DO PAGAMENTO']
Clausula: <completa>

Quem é o  

In [16]:
pdf_file_path = wd_path + "/data/1.2.5.17.10.1.1 CCB 511300712 - BB.pdf"

# Load and Parse PDFs

In [17]:
from llama_parse import LlamaParse  # pip install llama-parse

# parser = LlamaParse(
#     # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
#     result_type="markdown"  # "markdown" and "text" are available
# )

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="pt",
    num_workers=2,
)

# sync
# documents = parser.load_data("./my_file.pdf")

# async
# docs0 = await parser.aload_data("../data/1.2.5.17.10.1.1 CCB 511300712 - BB.pdf")

# LlamaIndex Recursive Query Engine

## Extract documents

In [18]:
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core.indices.loading import load_index_from_storage
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
Settings.callback_manager = CallbackManager([llama_debug])

In [19]:
# "DISPOSIÇÕES APLICÁVEIS AOS CONTRATOS DO BNDES".lower()

In [20]:
PERSIST_DIR = wd_path + "/data/storage"

os.path.exists(PERSIST_DIR)

True

In [21]:
# check if storage already exists

get_from_storage = os.path.exists(PERSIST_DIR)
# get_from_storage = False

# node_parser_type = "markdown"
node_parser_type = "metadata"
if not get_from_storage:
    # load the documents and create the index
    # documents = SimpleDirectoryReader(llamaindex_dir + "/data").load_data()
    documents = await parser.aload_data(pdf_file_path)
    if node_parser_type == "markdown":
        node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo"), num_workers=8)
        nodes = node_parser.get_nodes_from_documents(documents=[documents[0]])
        base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
        nodes_to_index = base_nodes+objects
    if node_parser_type == "metadata":
        pipeline = get_metadata_node_parser_pipeline()
        nodes_to_index = pipeline.run(
            documents=documents,
            # nodes=base_nodes + objects,
            in_place=True,
            show_progress=True,
        )
    # Excluding metadata from text to be embedded
    for n in nodes_to_index:
        n.excluded_embed_metadata_keys = ["questions_this_excerpt_can_answer", "prev_section_summary", "section_summary"]
    recursive_index = VectorStoreIndex(nodes=nodes_to_index)
    # index = VectorStoreIndex.from_documents(documents)
    # store it for later
    # saving the embeddings to disk
    # By default, this will save the data to the directory storage, but you can change that by passing a persist_dir parameter.
    recursive_index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    recursive_index = load_index_from_storage(storage_context)

**********
Trace: index_construction
**********


In [22]:
# from llama_index.core.schema import MetadataMode


# n_index = 0
# nodes_to_index[n_index].excluded_embed_metadata_keys = ["questions_this_excerpt_can_answer", "prev_section_summary", "section_summary"]
# nodes_to_index[n_index]

# print(nodes_to_index[n_index].get_content(metadata_mode=MetadataMode.EMBED))

In [23]:
# recursive_index = VectorStoreIndex(nodes=nodes_to_index)
# recursive_index.storage_context.persist(persist_dir=PERSIST_DIR)

## Check metadata and content

In [24]:
# UPDATE DOCUMENTS WITHOUT HAVING TO PARSE AGAIN
# for n in nodes_to_index:
#     n.excluded_embed_metadata_keys = ["questions_this_excerpt_can_answer", "prev_section_summary", "section_summary"]

In [71]:
# n_index = 5
# print(nodes_to_index[n_index].get_content(metadata_mode=MetadataMode.EMBED))
# print(nodes_to_index[n_index].get_content(metadata_mode=MetadataMode.LLM))

In [26]:
# print(objects[1].metadata["col_schema"])
# print(base_nodes[1].metadata)
# nodes_with_metadata[0].metadata
# TODO: add metadata to these base nodes

In [70]:
# duvidas["Data de Assinatura"][current_type]

In [69]:
# nodes_with_metadata[0].metadata["questions_this_excerpt_can_answer"]
# nodes_to_index[0].metadata

# for node in nodes_to_index:
#     if "ribeirao" in node.get_content().lower():
#         print("\n\n-------------------")
#         print(node.get_content())
#         print(node.id_)

In [112]:

# print_nodes_parsing_result(base_nodes)
# print_nodes_parsing_result(objects, "embed")
# print_nodes_parsing_result(objects, "llm")
# print_nodes_parsing_result(nodes_with_metadata, "embed")

# ids_list = ["b7d05e4d-21d4-44a5-90a8-a1a188b20072", "16966cdb-8c94-40ae-8ab7-4719558a0045"]
# ids_list = ["956167cb-3426-4b92-a71c-5a2e725eb167", "ea2982c0-4153-448c-9923-a0bd4181c06a"]
ids_list = ["ea2927aa-13e7-429f-ab2e-62441f32e654"]
print_nodes_parsing_result(nodes_to_index, metadata_mode="all", only_metadata=True, ids_list=ids_list)
# SEEMS LIKE THE IDEA OF RECURSIVE RETRIEVER IS:
# - EMBEDDING RETRIEVAL: WITHOUT THE TABLE NODES CONTENT
# - LLM INTERPRETATION: WITH THE TABLE NODES CONTENT

length:  28


----------------------- ea2927aa-13e7-429f-ab2e-62441f32e654 -----------------------
questions_this_excerpt_can_answer: 1. Qual é o valor da Cédula de Crédito Bancário emitida pela ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA em favor do Banco do Brasil S.A.?
2. Qual é a data de vencimento final da Cédula de Crédito Bancário?
3. Onde devem ser cumpridas as obrigações assumidas nesta Cédula de Crédito Bancário?

Resumo de nível mais alto: Este contexto fornece informações sobre uma Cédula de Crédito Bancário emitida pela ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025. O documento foi aditado, retificado e ratificado, e as obrigações devem ser cumpridas na Agência do Banco do Brasil S.A. em Ribeirão Preto-SP.
prev_section_summary: - Como solicitar correções ou exclusões de informações no SCR
- Números de telefone para entrar em contato com o Banco para sugestões, reclamações ou escla

In [19]:
# print(documents[0].text[:500])

# ## CEDULA DE CREDITO BANCARIO

# INTRODUÇÃO: Nr. 511.300.712

# 1. EMITENTE:

# |Razão ou denominação social:|ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA|
# |---|---|
# |CNPJ:|55.979.736/0001-45|
# |Conta Corrente:|000.003.203-4|
# |Endereço:|ROD ABRAÃO ASSED S/N KM 53 MAIS 450 METROS, RECREIO ANHANGUERA|
# |Cidade:|RIBEIRÃO PRETO-SP|
# |CEP:|14.097-500|
# |E-MAIL:|fabiola.furtadoCalliage-global.com|

# 2. DADOS DA OPERAÇÃO DE CRÉDITO:

# |Valor:|R$25.000.000,00 (vinte e cinco milhões de reais)|
# |---|---|
# |Vencimento:|1

In [ ]:
# from IPython.display import Markdown, display

# # define prompt viewing function
# def display_prompt_dict(prompts_dict):
#     for k, p in prompts_dict.items():
#         text_md = f"**Prompt Key**: {k}" f"**Text:** "
#         display(Markdown(text_md))
#         print(p.get_template())
#         display(Markdown(""))
# prompts_dict = query_engine_base.get_prompts()

# display_prompt_dict(prompts_dict)

## Retriever

- [Node Postprocessor](https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/)

In [28]:
from duvidas import duvidas

duvida = duvidas["Data de Assinatura"][current_type]

In [29]:
debug_post_process = False
show_retrievals = False
if debug_post_process:
    from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

    base_retriever = recursive_index.as_retriever(similarity_top_k=15)
    reranker = FlagEmbeddingReranker(
        top_n=5,
        model="BAAI/bge-reranker-large",
    )
    if show_retrievals:
        retrievals = processor.postprocess_nodes(base_retriever.retrieve(duvida))
else:
    base_retriever = recursive_index.as_retriever(similarity_top_k=2)
    if show_retrievals:
        retrievals = base_retriever.retrieve(duvida)

In [30]:
if show_retrievals:
    for n in retrievals:
        display_source_node(n, source_length=5000)

## Query engine

### Base

In [39]:
from duvidas import duvidas
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, verbose=True)

In [108]:
%%time

from duvidas import duvidas

recursive_search = False
# recursive_search = True
not_to_search = [
    # DONE
    "Objeto", "Data de Assinatura", "Alteração de Controle/ Reorganização", "Cessão", 
     # TODO LATER
     "Observações"]

# to_search = ["Garantias"]
# to_search = ["Cross default"]
# to_search = ["Objeto", "Cessão"]
# to_search = ["Alteração de Controle/ Reorganização"]
# to_search = ["Data de Assinatura"]
to_search = ["Devedor"]
# current_type = "Contrato de Financiamento"
current_type = "Cédula de Crédito Bancário"
responses = {}

original_stdout = sys.stdout
with open(wd_path + '/data/query_engine_output.txt', 'w') as f:
    # Redirect stdout to the file
    sys.stdout = f
    
    for col_name, duvida_map in duvidas.items():
        # if col_name not in not_to_search:
        responses[col_name] = dict()
        if col_name in to_search:
            print("\n\n\n------------- " + col_name.upper() + "-------------\n")
            if recursive_search:
                responses[col_name]["text"] = recursive_query_engine.query(
                    duvidas[col_name][current_type]
                )
            else:
                responses[col_name]["text"] = query_engine_base.query(
                    duvidas[col_name][current_type]
                )
    # Restore the original stdout
    sys.stdout = original_stdout
# 17 seconds

CPU times: user 35.4 ms, sys: 18.8 ms, total: 54.1 ms
Wall time: 2.07 s


In [109]:
for col_name, response in responses.items():
    print("\n\n\n------------- " + col_name.upper() + "-------------\n")
    print(response)

# ------------- DATA DE ASSINATURA-------------

# A data de assinatura, ou data de emissão, é 15/08/2025.
# O documento foi assinado em 18 de novembro de 2020.




------------- DEVEDOR-------------

O devedor (emitente) é a ALLIAGE S/A Indústrias Médico Odontológica.


In [86]:
# Find the retrieved nodes
# for n in responses["Devedor"].source_nodes:
#     display_source_node(n, source_length=5000)

In [110]:
from llamaindex_utils import *
from duvidas import *

for col_name, duvida_map in duvidas.items():
    # if col_name not in not_to_search:
    if col_name in to_search:
        print("\n\n\n------------- " + col_name.upper() + "-------------\n")
        responses[col_name]["clausula"] = await _agenerate_all_nodes_llm_answer(responses[col_name].source_nodes, current_type, duvida_map[current_type])
        # print(answer)
# assistant: AVALISTA(S)




------------- DEVEDOR-------------

** Messages: **
user: Uma seção deste documento foi dedicada a responder à seguinte dúvida: ```Identifica o  Devedor (Emitente)```.

Aqui está o conteúdo desta seção:
```PARÁGRAFO ÚNICO - Qualquer recebimento das prestações Tora dos prazos avencados constituira mera Loleranci 3, que nao dos afetaraprazosde forma algumaavençadosconstituirá mera tolerância, que não as datas de seus vencimentos ou as afetará declausulasforma alguma as datas de seus»-venctimentós ou as dema is condicoes deste Instrumento nem demais cláusulas e condições deste Instruífénto, nem importara novacao ou modificacao do ajustado, inclusive importará novação ou modificação do quanto encargos resultantes rajustadê, inclusive ao5 da mora, imputando se ao quanto aos encargos resultantes dêã&mora, imputando-se pagamento do debito valor recebido obrigatoriamente ao pagamento do debito o valor recebido obrWgatoriamente na na seguinte ordem: multa juros moratorios juros seguinte orde

In [ ]:
# to_search = ["Alteração de Controle/ Reorganização"]
# assistant: CLAUSULA ESPECIAL DE VENCIMENTO ANTECIPADO: RIGHT
# IDEA: TO IMPROVE THE RESULT: try HierarchicalNodeParser with AutoMergingRetriever → so we search by the more granular texts (128 tokens and then 2048 tokens for llm interpretation)

In [56]:
# ------------- ALTERAÇÃO DE CONTROLE/ REORGANIZAÇÃO-------------

# WITH RECURSIVE: A cláusula de vencimento antecipado menciona que o Banco do Brasil S.A. poderá considerar vencidas antecipadamente as operações de crédito se houver inadimplemento ou se ocorrer a transferência do controle do capital direto e/ou indireto, bem como a substituição de acionistas que resulte em perda do controle societário.

# NO RECURSIVE: A cláusula de vencimento antecipado menciona que o banco pode considerar antecipadamente vencidas as operações de crédito se houver transferência de controle direto e/ou indireto do capital, alteração na titularidade do capital social envolvendo acionistas com variação superior a 10%, ou alteração societária que resulte em perda de controle societário. Também é destacado que qualquer modificação no estatuto ou nos dirigentes deve ser comunicada ao Banco do Brasil durante a vigência da operação de crédito.


### Recursive with reranking

In [71]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=15,
    node_postprocessors=[reranker],
    verbose=True
)

In [23]:
# !python --version
# Python 3.10.12

response = recursive_query_engine.query(
    # duvidas["Data de Assinatura"][current_type]
    duvidas["Alteração de Controle/ Reorganização"][current_type]
    )
# MIGHT HAVE OUT OF MEMORY ISSUES

In [22]:
print(response)
# A data de assinatura da Cédula de Crédito Bancário é 18 de novembro de 2020 em Ribeirão Preto, São Paulo.

# A Alteração de Controle/Reorganização mencionada na cláusula de vencimento antecipado da cédula de crédito bancário ocorre quando há a transferência do controle direto e/ou indireto do capital, alteração de titularidade do capital social com variação superior a 10%, ou mudança na estrutura societária que resulte em perda de controle societário.

Na cláusula de Alteração de Controle/Reorganização, consta que o banco poderá considerar antecipadamente vencidas as obrigações da operação de crédito se houver transferência do controle direto e/ou indireto do capital, alteração de titularidade do capital social envolvendo qualquer acionista com variação superior a 10% do capital social, ou alteração social de qualquer controlada que resulte em perda de controle societário. Além disso, a cláusula prevê a comunicação obrigatória ao Banco do Brasil em caso de modificação do estatuto ou dos dirigentes durante a vigência da operação de crédito.
